# Import Dependencies

In [1]:
import numpy as np
import pandas as pd
import altair as alt
import panel as pn
pn.extension('vega')

pn.__version__

'1.3.8'

# Load Data

In [2]:
# Load Data 数据导入 #
# Gender Equality Index 性别平等指数导入
index_file = pd.ExcelFile('index_file.xlsx')  # Update with the correct file path

# Creating a dictionary to hold dataframes
dfs = {}
for sheet_name in index_file.sheet_names:
    df = pd.read_excel(index_file, sheet_name=sheet_name, usecols=['Index year', 'Gender Equality Index', 'Country'])
    dfs[sheet_name] = df

# Merging dataframes and renaming columns
index_all = pd.concat(dfs.values(), axis=0)
index_all = index_all.rename(columns={'Index year': 'year', 'Gender Equality Index': 'Equality_Index'})
index_all['year'] = index_all['year'].astype(str)

# Ensure unique_years is defined
unique_years = sorted(index_all['year'].unique())

# Filtering EU data
index_eu = index_all[index_all['Country'] == 'EU']


In [3]:
import altair as alt
import pandas as pd
import panel as pn

# Activate the Panel extensions for Altair
pn.extension('vega')

# Assuming the data has been loaded into `index_all` as done previously
# Make sure to filter for EU only data for the line chart
index_eu = index_all[index_all['Country'] == 'EU']

# Find the min and max of the Gender Equality Index to set the y-axis domain for the EU data
min_index = index_eu['Equality_Index'].min()
max_index = index_eu['Equality_Index'].max()

# Create the selection interval for the EU index chart
brush = alt.selection_interval(encodings=['x'], name="brush")

# Create the EU index chart with interval selection
eu_index_chart = alt.Chart(index_eu).mark_line().encode(
    x='year:O',
    y=alt.Y('Equality_Index:Q', scale=alt.Scale(domain=(min_index-2, max_index+2))),
    tooltip=['year', 'Equality_Index']
).properties(
    title='EU Gender Equality Index Over Time',
    width=750,
    height=500
).add_selection(brush)

# Create the ranking chart that will show the average index per country with gradient colors
# The tooltip will display the average index and country
ranking_chart = alt.Chart(index_all).transform_filter(
    brush
).transform_aggregate(
    average_index='mean(Equality_Index)',
    groupby=['Country']
).mark_bar().encode(
    x=alt.X('average_index:Q', title='Average Gender Equality Index'),
    y=alt.Y('Country:N', sort=alt.EncodingSortField(field="average_index", order="descending"), title='Country'),
    color=alt.condition(
        alt.datum.Country == 'EU',  # If the country is EU
        alt.value('red'),  # Use the contrasting color for EU
        alt.Color('average_index:Q', scale=alt.Scale(scheme='blues'))  # Use a blue gradient for others
    ),
    tooltip=['Country:N', 'average_index:Q']
).properties(
    width=300,
    height=500,
    title='Gender Equality Index Ranking'
)

# Combine the two charts horizontally
combined_chart = alt.hconcat(eu_index_chart, ranking_chart).resolve_scale(
    color='independent'
)

# Display the chart in Panel
pn.panel(combined_chart).show()


Launching server at http://localhost:52659


/Users/wangjiuduo/anaconda3/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
